In [1]:
import requests
import json
import pandas as pd
import time
import re
import urllib
import matplotlib
import numpy as np
import requests
import json
pd.set_option('display.max_colwidth', None)

In [2]:
astro_token_addr = "terra1nsuqsk6kh58ulczatwev87ttq2z6r3pusulg9r24mfj2fvtzd4uq3exn26"
xastro_token_addr = "terra1x62mjnme4y0rdnag3r8rfgjuutsqlkkyuh4ndgex0wl3wue25uksau39q8"

In [3]:
import requests
import base64
import time

def get_addresses(token_address):
    # URL to call
    url = f"https://phoenix-lcd.terra.dev/cosmwasm/wasm/v1/contract/{token_address}/smart"
    last_address = None
    # Base64 encode the message
    init_message = '{"all_accounts":{"limit":30}}'
    addresses = []
    while True:
        time.sleep(1)
        cont_message = '{{"all_accounts":{{"limit":30,"start_after":"{}"}}}}'.format(last_address)
        if(last_address):
            encoded_message = base64.b64encode(cont_message.encode()).decode()
        else:
            encoded_message = base64.b64encode(message.encode()).decode()
        
        # Construct the headers with the base64 encoded message
        headers = {
            "Content-Type": "application/json"
        }
        
        url_ = f"{url}/{encoded_message}"
        #print(url_)
        # Make the GET request to the URL with the encoded message in the headers
        response = requests.get(url_, headers=headers)
        
        # Check the response
        if response.status_code == 200:
            try:
                data = response.json()
                json.dumps(data, indent=2)
                if(len(data['data']['accounts']) == 0):
                    break
                addresses = list(set(addresses + data['data']['accounts']))
                print(len(addresses))
                last_address = data['data']['accounts'][-1]
            except:
                break
        else:
            print("Request failed. Status code:", response.status_code)
            print("Response text:", response.text)
    return addresses

astro_addr = get_addresses(astro_token_addr)

xastro_addr = get_addresses(xastro_token_addr)

xastro_addr_df = pd.DataFrame(xastro_addr, columns=['address'])
xastro_addr_df.to_csv("./data/astro/xastro_addresses.csv", index=False)
astro_addr_df = pd.DataFrame(astro_addr, columns=['address'])
astro_addr_df.to_csv("./data/astro/astro_addresses.csv", index=False)

In [6]:
astro_addr_df = pd.read_csv("./data/astro/astro_addresses.csv")
xastro_addr_df = pd.read_csv("./data/astro/xastro_addresses.csv")

In [7]:
import requests
import json

def get_balances(address):
    url = "https://starscream-terra-mainnet.chainscope.dev/"
    headers = {
        "Content-Type": "application/json"
    }
    
    body = {
        "query": """
        query Query($address: String!, $tokenContracts: [String!]!) {
            tokenBalances(address: $address, tokenContracts: $tokenContracts)
        }
        """,
        "variables": {
            "address": f"{address}",
            "tokenContracts": [
                "terra1x62mjnme4y0rdnag3r8rfgjuutsqlkkyuh4ndgex0wl3wue25uksau39q8",
                "terra1nsuqsk6kh58ulczatwev87ttq2z6r3pusulg9r24mfj2fvtzd4uq3exn26"
            ]
        },
        "operationName": "Query"
    }
    
    response = requests.post(url, headers=headers, data=json.dumps(body))
    
    if response.status_code == 200:
        result = response.json()
        return result
    else:
        print("Request failed with status code:", response.status_code)

In [8]:
balances = {}

In [9]:
i = 0
for address in pd.concat([xastro_addr_df, astro_addr_df]).address.unique():
    i += 1
    if(not address in balances):
        balances[address] = get_balances(address)['data']['tokenBalances']
        time.sleep(0.1)
    if(i % 1000 == 0):
        print(i)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [18]:
df = pd.DataFrame(balances, index=['xASTRO','ASTRO'])
df = df.T.reset_index()
df.columns = ['address','xASTRO','ASTRO']
df.to_csv('./data/astro/balances_20230720.csv', index=False)

In [33]:
df['xASTRO'] = df['xASTRO'].apply(lambda x: int(x)/1000000)
df['ASTRO'] = df['ASTRO'].apply(lambda x: int(x)/1000000)
df['total_astro'] = (df['xASTRO'] + df['ASTRO'])

In [36]:
df.to_csv('./data/astro/balances_20230720_polished.csv', index=False)

In [37]:
df

,address,xASTRO,ASTRO,total_astro
0,terra1rva5espyu5jdtp605vueea080rdlhyavvdnyyh,0.00000,24.847662,24.847662
1,terra1g8zgqc07rfhj6nycez0hku9uscw30jf2pzhxlx,1309.43102,3.636197,1313.067217
2,terra1nwhm8fm29zy9yge92atvs9zua6yg5eaucgf6x3,98.44090,0.000000,98.440900
3,terra130hnn9qh9naezd8q9p5cext6qse35ktxpqe07d,0.00000,50.231731,50.231731
4,terra169hpn0fr2tf5un4rknndttncveshqkkq3yp6x8,13466.32811,0.000415,13466.328525
...,...,...,...,...
19907,terra1walzuglwr9f9ytf68gke0gpvtdsqdvdv4d0hwl,0.00000,0.000000,0.000000
19908,terra1fa0fj6cpqkkxx3jne7wxgvju24kpzy2h046h35,0.00000,0.000000,0.000000
19909,terra1wcyu95j3ujuu4m7luve6f0pp57mzsmgsqxjy50,0.00000,430.696380,430.696380
19910,terra1ujs2t68pgk79wy3arzckyz6m5etvvcvp8zv5dm,0.00000,39.529769,39.529769
